# Expected Completion

### Conor Malone 14/05/2023

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt #plot AUC
from scipy.spatial.distance import cdist #function to get distances
import xgboost as xgb #gradient boosted decision tree classification method
from sklearn.metrics import accuracy_score, precision_score, recall_score, precision_recall_curve, f1_score, classification_report, mean_squared_error, log_loss,confusion_matrix, roc_curve, auc #metrics to assess model

In [2]:
plays = pd.read_csv("data/pass_receiver.csv")
pff = pd.read_csv("data/pffScoutingData.csv")
games = pd.read_csv("data/games.csv")

read in weekly tracking files and concatenate into one tracking file, then delete weeklies

In [3]:
week1 = pd.read_csv("data/week1.csv")
week2 = pd.read_csv("data/week2.csv")
week3 = pd.read_csv("data/week3.csv")
week4 = pd.read_csv("data/week4.csv")
week5 = pd.read_csv("data/week5.csv")
week6 = pd.read_csv("data/week6.csv")
week7 = pd.read_csv("data/week7.csv")
week8 = pd.read_csv("data/week8.csv")
tracking = pd.concat([week1,week2,week3,week4,week5,week6,week7,week8])


In [4]:
#save memory
del week1,week2,week3,week4,week5,week6,week7,week8

In [5]:
plays = plays.merge(games[["gameId","homeTeamAbbr"]], on = "gameId")
plays["possTeamScoreDiff"] = np.where(plays["homeTeamAbbr"]==plays["possessionTeam"],plays["preSnapHomeScore"]-plays["preSnapVisitorScore"],plays["preSnapVisitorScore"]-plays["preSnapHomeScore"])
plays["possTeamInLead"] = np.where(plays["possTeamScoreDiff"]>0,1,0)

add unique identifier of each play and frame (some playIds repeat over games)

In [6]:
tracking["comb_id"] = tracking["gameId"].astype("str")+" "+tracking["playId"].astype("str")
pff["comb_id"] = pff["gameId"].astype("str")+" "+pff["playId"].astype("str")
plays["comb_id"] = plays["gameId"].astype("str")+" "+plays["playId"].astype("str")
tracking["comb_and_frame"] = tracking["gameId"].astype("str")+" "+tracking["playId"].astype("str")+" "+tracking["frameId"].astype("str")

isolate frames at time of pass

In [7]:
#isolate just forward passes and leave out a certain frame (duplicate forward pass for a play)
just_forward_passes = tracking[(tracking["event"]=="pass_forward") & (tracking["comb_and_frame"]!="2021091203 3740 32")& (tracking["team"]!="football")]
#bring in receiver info from plays
tracking_plays =just_forward_passes.merge(plays[["comb_id","possessionTeam","is_pass_failed","PassReceiver_nflId","down","yardsToGo","yardlineNumber","possTeamScoreDiff","possTeamInLead"]], how="left",on="comb_id" )
#and qb info from pff
tracking_merged_all = tracking_plays.merge(pff[["comb_id","nflId","pff_role"]], how = "left",on=["comb_id","nflId"])

# Function to take in 1 pass play and  format it

In [8]:
def one_play_function(data):
    one_play_formatted_list = pd.DataFrame()
    data = data.reset_index(drop=True)
    comb_id = data["comb_id"][0]
    #get y, the response variable for this play (binary pass completed or not) and possesionteam (for CV)
    y = data["is_pass_failed"][0]
    team = data["possessionTeam"][0]
    QB = data[data["pff_role"]=="Pass"].head(1)
    WR = data[data["nflId"]==data["PassReceiver_nflId"]]
    #if no clear receiver just pick anyone
    if len(WR)==0:
        WR = data[data["pff_role"]=="Pass Route"].head(1)
 
    qb_loc = QB[["x","y"]]
    wr_loc = WR[["x","y"]]
    coverage_loc = data[["x","y"]][data["pff_role"]=="Coverage"]
    rush_loc = data[["x","y"]][data["pff_role"]=="Pass Rush"]
    #put play in adjacency matrix format, the distance of all players to each other (this will then be filtered to just link relevant players to QB or WR)
    QB_dist = min((i for i in cdist(qb_loc.values,wr_loc.values)[0] if i is not None),default=[])
    Coverage_dist = min((i for i in cdist(coverage_loc.values,wr_loc.values)[0] if i is not None),default=[])
    if len(rush_loc)==0:
        Rush_dist = None
    else:
        Rush_dist = min((i for i in cdist(rush_loc.values,qb_loc.values)[0] if i is not None),default=[])#min((i for i in cdist(rush_loc.values,qb_loc.values)[0] if i is not None),default=[])
    #then put play in features format, getting a table containing position, speed, orientation etc
    rel_o = WR["o"].iloc[0]-QB["o"].iloc[0]
    rel_dir = WR["dir"].iloc[0]-QB["dir"].iloc[0]
    wr_s = WR["s"]
    qb_s = QB["s"]
    one_play = pd.DataFrame({"comb_id":comb_id,"y":y,"Team":team,"QB_Dist":QB_dist,"DB_Dist":Coverage_dist,"Rush_Dist":Rush_dist,"Orientation":rel_o,"Direction":rel_dir,"WR_Speed":WR["s"],"QB_Speed":QB["s"],"down":WR["down"],"yardsToGo":WR["yardsToGo"],"yardLine":WR["yardlineNumber"],"ScoreDiff":WR["possTeamScoreDiff"],"inLead":WR["possTeamInLead"]})
    
    
    one_play_formatted_list = pd.concat([one_play_formatted_list,one_play], ignore_index = True)
    return one_play_formatted_list

run function on each play in dataset

In [9]:
table_list = pd.DataFrame()
for i in range(len(tracking_merged_all.comb_id.unique())):    
    one_play = tracking_merged_all[tracking_merged_all.comb_id==tracking_merged_all.comb_id.unique()[i]]
    one_play = one_play.reset_index(drop=True)
    result = one_play_function(one_play)
    table_list = pd.concat([table_list,result],ignore_index=True)


workaround to stop adding duplicate cols when qb_speed is included

In [10]:

qb_speed = table_list[["comb_id","QB_Speed"]][~table_list.QB_Speed.isna()]
tab_2 = table_list[table_list.QB_Speed.isna()].drop("QB_Speed", axis=1)
tab_2 = tab_2.merge(qb_speed, on = "comb_id")
table_list = tab_2

Cross Validation with k = 32 by loop through and for each team use as val (remove from training) and store predictions

In [36]:
# Define the XGBoost model
model = xgb.XGBClassifier()
unique_team_list = table_list.Team.unique()
f1_list = []
acc_list = []
mse_list = []
ll_list = []
all_prediction_list = []
#cross validate leaving out each team once
for i in range(len(unique_team_list)):
    train = table_list[table_list["Team"]!=unique_team_list[i]]
    val = table_list[table_list["Team"]==unique_team_list[i]]
    x_train = np.array(train.loc[:, "QB_Dist":"QB_Speed"])
    y_train = np.array(train.loc[:,"y"])
    model.fit(x_train, y_train)
    val_predictions = model.predict(np.array(val.loc[:, "QB_Dist":"QB_Speed"]))
    predictions = model.predict(np.array(table_list.loc[:, "QB_Dist":"QB_Speed"]))
    val["pred"] = val_predictions.tolist()
    y_val = val["y"]
    y_val_pred = np.where(val["pred"]>.5,1,0)
    f1 = f1_score(y_val, y_val_pred, average='binary')
    accuracy = accuracy_score(y_val, y_val_pred)
    mse = mean_squared_error(y_val, y_val_pred)
    ll = log_loss(y_val, y_val_pred)
    all_prediction_list.append(predictions)
    f1_list.append(f1)
    acc_list.append(accuracy)
    mse_list.append(mse)
    ll_list.append(ll)

C:\Users\conny\AppData\Local\Temp\ipykernel_16108\585828426.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val["pred"] = val_predictions.tolist()
C:\Users\conny\AppData\Local\Temp\ipykernel_16108\585828426.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val["pred"] = val_predictions.tolist()
C:\Users\conny\AppData\Local\Temp\ipykernel_16108\585828426.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

get average prediction for each play from the 32 runs

In [37]:
np.mean(mse_list, axis=0)

0.3355471800536436

In [38]:
np.mean(acc_list, axis=0)

0.6644528199463564

In [39]:
np.mean(ll_list, axis=0)

11.589492006895094

In [40]:
np.mean(f1_list, axis=0)

0.5202853145877626